## 📌 About This Notebook

This notebook is a **continuation of Video 06: Temperature Trends in Egypt (2000–2024)**.  
If you haven’t watched or run that notebook yet, you can check it here:

🎥 **Video 06 on YouTube**  
[Google Earth Engine with Python [06] - Egypt Monthly Temperature Time Series (2000–2024) [Arabic]](https://www.youtube.com/watch?si=B3JqNh_F1RMEEmyI&v=h9wUak_YTeM&feature=youtu.be)

📄 **Code for Video 06 on GitHub**  
[06_Egypt_Monthly_Temperature_Time_Series & Annual_Maps (2000–2024).ipynb](https://github.com/NourNegm85/GEE-Tutorials-With-Python/blob/main/notebooks/06_Egypt_Monthly_Temperature_Time_Series%20%26%20Annual_Maps_(2000-2024).ipynb)


##🎯 What You'll Learn in This Video

✅ Calculate Sen’s slope for temperature trends in Egypt (2000–2024)

✅ Visualize change per year and total change over the study period

In [ ]:
# Install xee library
! pip install xee pymannkendall

### 📌what is Sen's Slope? and why we use it

هو طريقة احصائية لحساب معدل التغير في القيم على مدى زمني في السلاسل الزمنية

🔹 لماذا نستخدمه؟  
- يعطي نتيجة أكثر دقة في حالة البيانات المتقلبة أو غير المنتظمة.  
- لا يتأثر كثيرًا بالقيم الشاذة (Outliers).  
- مناسب جدًا لدراسة تغيّر المناخ أو أي اتجاه طويل المدى.

📌 في هذا الفيديو سنستخدمه لقياس معدل تغيّر درجة الحرارة في مصر من سنة 2000 إلى 2024، وسننشئ خريطة توضّح أماكن الارتفاع الأكبر في درجات الحرارة.


In [ ]:
# Import libraries
import ee
import geemap
import xarray as xr
import xee
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import plotly.express as px
import pandas as pd
import numpy as np
import pymannkendall as mk

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()

ee.Initialize(project = 'ee-nouribrahim25' , opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Define Egypt Region
egy = ee.FeatureCollection ('projects/ee-nouribrahim25/assets/EGY-ADM0').geometry()
egy

In [ ]:
# Load ERA5-Land Monthly temperature data (Kelvin)
temp = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
      .filterBounds(egy)
      .filterDate('2000-01-01' , '2024-12-31')
      .select('temperature_2m')
      )

temp

In [ ]:
# Load into xarray using xee
ds = xr.open_dataset(
    temp,
    engine="ee",
    geometry= egy,
    crs="EPSG:4326",
    scale= 0.11,
)
ds

In [ ]:
# Convert Kelvin to Celsius
ds['temperature_2m'] = ds['temperature_2m'] - 273.15
ds

Monthly Time Series Plot (Plotly)

In [ ]:
#Aggregate to monthly mean over Egypt
monthly_temp = ds['temperature_2m'].mean(dim=["lon", "lat"])
monthly_temp

In [ ]:
#Convert to DataFrame
temp_df = monthly_temp.to_dataframe().reset_index()
temp_df.head()

In [ ]:
 #Plot with Plotly
fig = px.line(
    temp_df, x="time", y="temperature_2m",
    title="Monthly Mean Temperature in Egypt (2000–2024)",
    labels={"temperature_2m": "Temperature (°C)", "time": "Date"},
    template="plotly_white"
)
fig.show()

In [ ]:
# Add 12-month rolling average
temp_df['rolling'] = temp_df['temperature_2m'].rolling(12).mean()

In [ ]:
# Add rolling mean trace and update layout
fig.add_scatter(
    x=temp_df['time'], y=temp_df['rolling'],
    mode='lines', name='12-Month Rolling Avg',
    line=dict(color='red')
)

fig.update_xaxes(
    dtick="M12",
    tickformat="%Y",
    tickangle=0
)
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)
fig.show()

Annual Resampling & Temperature Maps

In [ ]:
# Resample the data into one year interval
annual = ds.resample(time = 'YE').mean('time')
annual

In [ ]:
# plot the annual data on the map
annual['temperature_2m'].plot(
    figsize=(18, 14),
    x = 'lon' ,
    y = 'lat' ,
    col = 'time' ,
    col_wrap=5 ,
    robust = True ,
    cmap = 'jet',
    vmin = 18 ,
    vmax = 30 ,
);

Define function to apply Sen's slope from pymannkendall

In [ ]:
def sen_fun(image):
    clean = image[~np.isnan(image)]
    return mk.original_test(clean).slope

In [ ]:
# Apply Sen's slope calculation to each pixel
sen_slope_map = xr.apply_ufunc(
    sen_fun,
    annual['temperature_2m'],
    input_core_dims=[['time']],   # Apply along the time dimension
    output_dtypes=[float],
    vectorize=True,               # Apply to each pixel independently
    dask='allowed'                # Enables parallel computation
)

Plot Sen's slope map

In [ ]:
# Plot with Matplotlib
plt.figure(figsize=(6, 4))
sen_slope_map.plot(
    x='lon',
    y='lat',
    robust=True,
    cmap='RdBu_r',
    cbar_kwargs={'label': 'Temperature Change (°C/year)'},
    vmin = -abs(sen_slope_map.max()),
    vmax = abs(sen_slope_map.max()),
    )
plt.title("Sen's Slope of Annual Mean Temperature in Egypt (2000–2024)")
plt.xlabel('Lon')
plt.ylabel('Lat')
plt.show()

Total change map (°C)

In [ ]:
# Number of years in the dataset
n_years = len(annual.time)

# Plot total change
plt.figure(figsize=(6, 4))
(sen_slope_map * n_years).plot(
    x='lon',
    y='lat',
    cmap='RdBu_r',
    norm=TwoSlopeNorm(vmin=-1.4, vcenter=0, vmax=1.4),
    cbar_kwargs={'label': 'Total Temperature Change (°C)'}
)
plt.title("Total Temperature Change in Egypt (25 years)")
plt.show()